# Air Force Cadet Career Problem Walkthrough

This jupyter notebook demonstrates the implementation of the Air Force Cadet Career Problem (afccp) python module developed and maintained by Griffen Laird for his master's thesis. This notebook showcases the "CadetCareerProblem" class, which is a class object of these specific problem instances. This class pertains to specific instances of the AFSC matching problem (AFCCP) and there are numerous class methods pertaining to importing, exporting, solving, etc. Let's first discuss some preliminary material for this module.  

## Preliminaries

This module's main purpose is to automate as much of the matching process as possible. If we can focus on the general application of solving these problems, there is no reason that AFPC needs to spend so much time each year cleaning data and figuring out how to assign cadets to their AFSCs! That's where afccp comes in.

### Changing Directory

To keep things cleaner, I put all the executable files (this notebook included) in their own folder. When we run one of those files, the working directory is initially that subfolder and so we would only have access to the files and modules within that sub-folder. This isn't going to work, so we need to change the working directory.

In [1]:
import os

# Obtain initial working directory
dir_path = os.getcwd() + '/'
print('current working directory:', dir_path)

# Get main afccp folder path
index = dir_path.find('afccp') 
dir_path = dir_path[:index + 6]

# Update working directory
os.chdir(dir_path)
print('updated working directory:', dir_path)

current working directory: /Users/griff007/Desktop/Python Projects/afccp/afccp/executables/examples/
updated working directory: /Users/griff007/Desktop/Python Projects/afccp/


### globals.py

This script figures out the user's directory information and determines which python packages the user has access to. The majority of the packages I use are common python packages (pandas, numpy, matplotlib, etc.) but there are a few exceptions. The libraries pyomo (optimization package for python), sdv (synthetic data vault useful for generating realistic data --CTGAN is found here!), and sklearn (Sci-kit learn but it's version 0.0 since for whatever reason this is the one I found that performs MDS for the solution similarity plots) are some of the less common ones. This script checks these packages directly and prints out whether we have them or not.

In [2]:
from afccp.core.globals import *

Sensitive data folder found.
Pyomo module found.
SDV module found.
Sklearn Manifold module found.


We also print out whether or not we have the "Sensitive" data folder. Since I ultimately want to publish this module to GitHub, I have a separate folder that is only stored on my local machine that has the real cadet/AFSC data. When we commit things to GitHub, there is a "gitignore" file that prevents the sensitive data from being published. 